<a href="https://colab.research.google.com/github/oh5221/DA_P1/blob/main/%EC%9E%90%EC%A3%BC_%EB%B0%A9%EB%AC%B8%ED%95%98%EB%8A%94_%ED%98%B8%ED%85%94_%EC%A7%80%EB%8F%84%EC%97%90_%ED%91%9C%EA%B8%B0%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# CSV 파일을 cp949 인코딩으로 읽기
file_path = '/content/hotel_to_maps.csv'
df = pd.read_csv(file_path, encoding='cp949')

# 데이터 확인
df.head()

# 중복된 값을 제거
df_unique = df.drop_duplicates()

# 중복 제거된 데이터 확인
df_unique.head()


,AGE_RANGE,LDGS_ADDR,LDGS_NM,LDGS_ROAD_NM_ADDR,visit_count
0,20대,강원,쏠비치 호텔 삼척,강원 삼척시 수로부인길 453,25
1,20대,강원,소노벨 비발디파크 b c 구 오크 파인,강원 홍천군 서면 한치골길 264,12
2,20대,강원,롯데리조트 속초,강원 속초시 대포항길 186,9
3,20대,강원,하이원 팰리스호텔,강원 정선군 고한읍 고한7길 399,8
4,20대,강원,쏠비치 호텔 양양,강원 양양군 손양면 선사유적로 678,8


In [ ]:
import requests
import time
# Kakao API 설정
kakao_api_key = '25792c1e4e2087c6f7cd5cc838a7be3a'

# Kakao Local API를 이용한 지오코딩 함수
def kakao_geocode(address):
    url = 'https://dapi.kakao.com/v2/local/search/address.json'
    headers = {"Authorization": f"KakaoAK {kakao_api_key}"}
    params = {'query': address}
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        result = response.json()
        if result['documents']:
            x = result['documents'][0]['x']
            y = result['documents'][0]['y']
            return y, x
        else:
            return None, None
    else:
        return None, None

# 각 주소에 대해 지오코딩 수행
latitude = []
longitude = []

for addr in df_unique['LDGS_ROAD_NM_ADDR']:
    lat, lon = kakao_geocode(addr)
    latitude.append(lat)
    longitude.append(lon)
    time.sleep(0.5)  # API 요청 제한을 피하기 위해 지연

# 데이터프레임에 위도와 경도 열 추가
df_unique['latitude'] = latitude
df_unique['longitude'] = longitude


In [ ]:
# Folium 라이브러리를 사용하여 지도 생성
import folium

# 중심 위치 설정 (서울을 중심으로 설정)
map_center = [37.5665, 126.9780]

# Folium 지도 생성
hotel_map = folium.Map(location=map_center, zoom_start=12)

# 지도에 마커 추가
for idx, row in df_unique.iterrows():
    if not pd.isnull(row['latitude']) and not pd.isnull(row['longitude']):
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            popup=row['LDGS_NM'],
            tooltip=row['LDGS_ROAD_NM_ADDR']
        ).add_to(hotel_map)

# 지도 저장
hotel_map.save('/content/drive/MyDrive/1stProject/hotel_map.html')
